# Decision Tree mosquito classifiar

# 1. Import the dependencies.

In [ ]:
# This program uses the images of the detected mosquitoes because I saved them, detection was done on those and they were uploaded online to train
from PIL import Image
import cv2
from skimage.feature import local_binary_pattern, hog
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import random
import pandas as pd
import os
from typing import Tuple
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, multilabel_confusion_matrix
import pickle

# 2. Load images, image preprocessing, and image feature extraction
- find_label find the corresponding label for each mosquito image.
- paths_labels finds the paths of images.
- gray_scale convert images to grayscale.
- clahe applies adaptive histogram equalization.
- histo_ori_grds extracts Histogram Oriented Gradients (HOG).

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/mosquito_names.csv")
img_names = data["img_fName"]
img_names = list(img_names)
img_classes = data["class_label"]

def find_label(file_name):
    if file_name in img_names:
        index = img_names.index(file_name)
        return img_classes[index]

def paths_labels() -> Tuple:
    directory = "/content/drive/MyDrive/detected_mosquitoes/train_test_images"
    img_paths = []
    img_labels = []
    # Getting each image path from the dataset
    for file_name in os.listdir(directory):
        if file_name.endswith('.jpeg'):
            file_path = os.path.join(directory, file_name)
            img_paths.append(file_path)
            img_labels.append(find_label(file_name))

    return img_paths, img_labels

def gray_scale(img):  # For gray scale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img_gray

def clahe(gray_image):  # Applying CLAHE
    adaptive_histogram_equalization = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image_equalized = adaptive_histogram_equalization.apply(gray_image)
    return image_equalized

def histo_ori_grds(preprocessed_region):  # HOG features
    features = hog(preprocessed_region, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    return features

hogs = []
imgs, img_labels = paths_labels()

for i in range(len(imgs)):
  # Load the image
  image = cv2.imread(imgs[i])

  hog_image = histo_ori_grds(clahe(gray_scale(image)))
  hogs.append(hog_image.ravel())

# 3. Dataset splitting, training a model, and evaluating a model.
- Split the data into training and testing data
- Create Decision Tree model and fit it to the training data.
- Finally make predictions with the model.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(hogs, img_labels, test_size=0.2, random_state=42)

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

preds = tree.predict(X_valid)
print(y_valid)
print(preds)
print("Accuracy score: ", accuracy_score(y_valid, preds))
print("Confusion matrix: ", multilabel_confusion_matrix(y_valid, preds, labels=['albopictus', 'culex', 'anopheles', 'culiseta', 'japonicus/koreicus', 'aegypti']))

# Saving the model using pickel
det_tree_model = "det_tree_model.pkl"
with open(det_tree_model, 'wb') as file:
    pickle.dump(tree, file)

['culex', 'albopictus', 'albopictus', 'culex', 'culex', 'albopictus', 'culex', 'culex', 'culex', 'culex', 'albopictus', 'culex', 'albopictus', 'culex', 'culex', 'albopictus', 'culex', 'albopictus', 'albopictus', 'japonicus/koreicus', 'albopictus', 'albopictus', 'culex', 'albopictus', 'culex', 'culex', 'culex', 'culex', 'culex', 'culex', 'albopictus', 'culex', 'culex', 'culex', 'culex', 'albopictus', 'culex', 'culex', 'culex', 'culex', 'albopictus', 'culex', 'albopictus', 'albopictus', 'albopictus', 'culex', 'culex', 'culex', 'culex', 'albopictus', 'albopictus', 'culex', 'culex', 'albopictus', 'culex', 'albopictus', 'albopictus', 'albopictus', 'culex', 'albopictus', 'culex', 'albopictus', 'culex', 'culex', 'albopictus', 'culex', 'culex', 'albopictus', 'culex', 'albopictus', 'albopictus', 'albopictus', 'culex', 'albopictus', 'albopictus', 'culex', 'culex', 'culex', 'culex', 'albopictus', 'albopictus', 'albopictus', 'albopictus', 'albopictus', 'culex', 'albopictus', 'albopictus', 'albopic